In [3]:
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
url = "https://cgm.demo-de.cloud.c3.cgm.ag/cgmg3"
if os.name == 'nt':  # Windows
    executable_path = '.\\chromedriver.exe'
else:  # Mac or other Unix-like systems
    executable_path = './chromedriver'
service = Service(executable_path=executable_path)
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [3]:
driver.execute_script("""
    var cursorContainer = document.createElement('div');
    cursorContainer.id = 'custom-cursor-container';
    cursorContainer.style.position = 'absolute';
    cursorContainer.style.pointerEvents = 'none';
    cursorContainer.style.zIndex = '10000';
    cursorContainer.style.transform = 'rotate(-45deg)'; // Dreieck und Balken gemeinsam drehen
    document.body.appendChild(cursorContainer);

    var cursor = document.createElement('div');
    cursor.id = 'custom-cursor';
    cursor.style.width = '0';
    cursor.style.height = '0';
    cursor.style.borderLeft = '10px solid transparent';
    cursor.style.borderRight = '10px solid transparent';
    cursor.style.borderBottom = '20px solid white';
    cursor.style.position = 'relative'; // Innerhalb des Containers positioniert
    cursor.style.pointerEvents = 'none';
    cursor.style.zIndex = '10000';
    cursorContainer.appendChild(cursor);

    var cursorTail = document.createElement('div');
    cursorTail.id = 'cursor-tail';
    cursorTail.style.width = '2px'; // Breite des kleinen Balkens
    cursorTail.style.height = '12px'; // Länge des kleinen Balkens
    cursorTail.style.backgroundColor = 'white'; // Gleiche Farbe wie das Dreieck
    cursorTail.style.position = 'absolute';
    cursorTail.style.top = '18px'; // Unterhalb des Dreiecks platzieren
    cursorTail.style.left = '8px'; // Leicht zur Seite versetzen
    cursorContainer.appendChild(cursorTail);

    var cursorAni = cursorContainer.cloneNode(true);
    cursorAni.id = 'animation-cursor';
    document.body.appendChild(cursorAni);

    document.addEventListener('mousemove', function(e) {
        cursorAni.style.left = e.pageX + 'px';
        cursorAni.style.top = e.pageY + 'px';
    });

    window.hideCursor = function() {
        cursorContainer.style.opacity = '0.0';
        document.body.style.cursor = 'none';
    }

    window.showCursor = function() {
        cursorContainer.style.opacity = '1';
        document.body.style.cursor = 'default';
    }
""")


In [ ]:
action = ActionChains(driver)

# Aktuelle Cursor-Position
current_x = 0
current_y = 0

# Funktion zum langsamen Bewegen des "Cursors" zwischen zwei Punkten
def move_cursor_to(target_element):
    global current_x, current_y

    # Zielposition des Elements bestimmen
    target_location = target_element.location
    target_size = target_element.size
    target_x = target_location['x'] + target_size['width'] / 2
    target_y = target_location['y'] + target_size['height'] / 2

    # Rote Punktbewegung simulieren
    driver.execute_script("window.hideCursor();")
    steps = 20  # Anzahl der Schritte für die Bewegung
    for step in range(steps + 1):
        # Berechnung der Zwischenposition (Interpolation)
        x = current_x + (target_x - current_x) * step / steps
        y = current_y + (target_y - current_y) * step / steps

        # Cursor-Position aktualisieren
        driver.execute_script(f"document.getElementById('animation-cursor').style.left = '{x}px';")
        driver.execute_script(f"document.getElementById('animation-cursor').style.top = '{y}px';")

        sleep(0.05)  # Zeit zwischen den Schritten

    # Nach dem Bewegen klicken
    action.click(target_element).perform()
    sleep(0.5)

    # Aktualisiere die aktuelle Cursor-Position nach dem Klick
    current_x = target_x
    current_y = target_y
    driver.execute_script("window.showCursor();")

# Zuerst auf "Menü" klicken, das ein <a>-Element ist
menuElement = driver.find_element(By.XPATH, "//a[contains(text(), 'Menü')]")
move_cursor_to(menuElement)

# Menü erneut anklicken, um es wieder einzuklappen
move_cursor_to(menuElement)

# Dann auf "Fallsuche" klicken
fallsucheElement = driver.find_element(By.XPATH, "//label[contains(text(), 'Fallsuche')]")
move_cursor_to(fallsucheElement)

# Zuletzt auf "Übersicht Arzt" klicken
uebersichtArztElement = driver.find_element(By.XPATH, "//label[contains(text(), 'Übersicht Arzt')]")
move_cursor_to(uebersichtArztElement)
